# Notebook Title

## Setup Python and R environment
you can ignore this section

In [17]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [19]:
%%R

# My commonly used R imports

require('tidyverse')

## Load & Clean Data

👉 Load the data along with the census connectors below (the output of the `connect-to-census.ipynb` notebook) and do any cleanup you'd like to do.

In [20]:
geocode_df=pd.read_csv("geocoded_schools.csv")

## 👉 Grab Census Data

1. loading the Census API key

In [21]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()


True

In [22]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("CENSUS_API_KEY"))

To install your API key for use in future sessions, run this function with `install = TRUE`.


2. Decide which Census variables you want

    Use <https://censusreporter.org/> to figure out which tables you want. (if censusreporter is down, check out the code in the cell below)

    -   Scroll to the bottom of the page to see the tables.
    -   If you already know the table ID, stick that in the "Explore" section to learn more about that table.

    By default this code loads (B01003_001) which we found in censusreporter here: https://censusreporter.org/tables/B01003/

    - find some other variables that you're also interested in
    - don't forget to pick a geography like "tract", "county" or "block group". here is the list of [all geographies](https://walker-data.com/tidycensus/articles/basic-usage.html#geography-in-tidycensus
    ).


In [23]:
%%R 

# Finding the Census Varaibles for the ACS 5 year survey
# Generall you'd do this in CensusReporter, but since it's down sometimes, here it is using tidycensus's load_variables function

# get every single variable in the ACS5 
# RR: Changed to acs1, changed year
all_census_vars <- load_variables(2022, "acs1", cache = TRUE) 

filtered_census_vars <- all_census_vars %>% 
    filter(grepl("median income", label, ignore.case = TRUE))   # filter to those containing "median income"
    
# write to CSV so we can view it in python
filtered_census_vars %>% 
    write_csv("filtered_census_vars.csv")

# show the first few rows
filtered_census_vars %>%
    #select(-geography) %>% # remove the geography column
    print(n = 20) # print the first 20 rows

# A tibble: 46 × 3
   name         label                                                    concept
   <chr>        <chr>                                                    <chr>  
 1 B06011PR_001 Estimate!!Median income in the past 12 months --!!Total: Median…
 2 B06011PR_002 Estimate!!Median income in the past 12 months --!!Total… Median…
 3 B06011PR_003 Estimate!!Median income in the past 12 months --!!Total… Median…
 4 B06011PR_004 Estimate!!Median income in the past 12 months --!!Total… Median…
 5 B06011PR_005 Estimate!!Median income in the past 12 months --!!Total… Median…
 6 B06011_001   Estimate!!Median income in the past 12 months --!!Total: Median…
 7 B06011_002   Estimate!!Median income in the past 12 months --!!Total… Median…
 8 B06011_003   Estimate!!Median income in the past 12 months --!!Total… Median…
 9 B06011_004   Estimate!!Median income in the past 12 months --!!Total… Median…
10 B06011_005   Estimate!!Median income in the past 12 months --!!Total… Median…
11 B07011

In [48]:
%%R -o nyc_census_data
# the variable B01003_001E was selectd from the census table 
# for population, which we found in censusreporter here:
# https://censusreporter.org/tables/B01003/

# in the table below, pick the geography, the variables, and the survey you want to pull from
# see the possible values here https://walker-data.com/tidycensus/articles/basic-usage.html

# Get variable from ACS
nyc_census_data <- get_acs(geography = "tract", 
                      state='NY',
                      county = c("New York", "Kings", "Queens", "Bronx", "Richmond"),
                      variables = c(
                        population="B01003_001",
                        med_inc="B19013_001",
                        school_kids="B14001_002",
                        kids="B09001_001",
                        white_race="B02001_002",
                        black_race="B02001_003",
                        asian_race="B02001_005",
                        male_public_school="B14003_003",
                        male_private_school="B14003_012",
                        female_public_school="B14003_031",
                        female_private_school="B14003_040"
                      ), 
                      year = 2022,
                      survey="acs5",
                      geometry = F)

nyc_census_data

nyc_census_data %>% 
    write_csv("nyc_census_data.csv")


Getting data from the 2018-2022 5-year ACS
Using FIPS code '36' for state 'NY'
Using FIPS code '061' for 'New York County'
Using FIPS code '047' for 'Kings County'
Using FIPS code '081' for 'Queens County'
Using FIPS code '005' for 'Bronx County'
Using FIPS code '085' for 'Richmond County'


## 👉 Merge it with your data

hint...`tidycensus` provides you data in long format you may need to pivot the census data from long to wide format before merging it with your data

In [60]:
%%R -i nyc_census_data
#drop moe column   
nyc_census_data <- nyc_census_data %>%
  select(-moe)

#pivot the data to wide format  (one row per tract)
nyc_census_data_wide <- nyc_census_data %>%
  pivot_wider(names_from = "variable", values_from = "estimate")
nyc_census_data_wide
#output the data to a csv file
nyc_census_data_wide %>%
  write_csv("nyc_census_data_wide.csv")

geo_code <- read.csv("geocoded_schools.csv")

# Merge the dataframes
merged_data <- merge(nyc_census_data_wide, geo_code, by = "GEOID", all.x = TRUE)
merged_data %>% 
    write_csv("merged_data.csv")

In [61]:
df=pd.read_csv("merged_data.csv")
df.dtypes

GEOID                      int64
NAME                      object
population                 int64
white_race                 int64
black_race                 int64
asian_race                 int64
kids                       int64
school_kids                int64
male_public_school         int64
male_private_school        int64
female_public_school       int64
female_private_school      int64
med_inc                  float64
Borough                   object
Community.Board          float64
School.Type               object
School.Name...ID          object
Address                   object
Postcode                 float64
Latitude                 float64
Longitude                float64
Council.District         float64
Census.Tract             float64
BIN                      float64
BBL                      float64
NTA                       object
STATE                    float64
COUNTY                   float64
TRACT                    float64
BLOCK                    float64
dtype: obj